In [2]:
import os
import re
import folium
import pandas as pd
import numpy as np
import datetime
from urllib.request import urlretrieve
from tqdm import tqdm_notebook as tqdm
# from ..hurricane_functions import *
from mpl_toolkits.basemap import Basemap

## Resources:
https://www.kaggle.com/poonaml/last-cab-to-new-york-animated-heatmap-trips-folium/

https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/

## Load Data

In [3]:
def read_hurdat(url, local_fname, location):
    if not os.path.exists(local_fname):
        urlretrieve(url, local_fname)

    records = []
    with open(local_fname,'r') as f:
        for line in f:
            if line.startswith(location):
                record = line.strip()
                reports = []
                records.append((record, reports))
            else:
                reports.append(line.strip())
                
    return records

In [4]:
url = "https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2017-050118.txt"
local_fname = "../data/hurdat2.txt"

records = read_hurdat(url, local_fname, "AL") # AL for atlantic hurricanes

## Analyze hurricanes data

In [5]:
# Define all functions required for parsing the raw data and mapping hurricane paths

def get_all_hurricanes(records):
    """
    The raw records from hurdat2 is a list of lists. We want 
    to parse this into a neat dataframe for easy analysis.
    """
    hurr_list = []
    for hurricane in tqdm(records, total=len(records)):
        year = hurricane[0].split(',')[0][-4:]
        hurr_id = hurricane[0].split(',')[0]
        hurr_list.append(hurr_id)
            
    num_hurr = len(hurr_list)
    records_for_yr = records[-num_hurr:]
    
    return records_for_yr

def create_hurricane_df(records):
    """
    The raw data from NOAA is a text file. This function parses that
    text file into a nice, readable Pandas dataframe for easy manipulation.
    """
    records_df = pd.DataFrame()
    idx = 0

    for record in records:
        hurricane_id = record[0].split(',')[0]
        hurricane_name = record[0].split(',')[1].strip()

        for datapoint in record[1]:
            data_list = [x.strip() for x in datapoint.split(',')]
            record_date = data_list[0]
            time = data_list[1]
            storm_status = data_list[3]
            lat = data_list[4]
            lon = data_list[5]
            max_wind = data_list[6]
            min_pressure = data_list[7]

            # Add to df
            records_df.loc[idx, 'id'] = hurricane_id
            records_df.loc[idx, 'name'] = hurricane_name
            records_df.loc[idx, 'date'] = record_date
            records_df.loc[idx, 'time'] = time
            records_df.loc[idx, 'dt'] = datetime.datetime.strptime(record_date+time,'%Y%m%d%H%M')        
            records_df.loc[idx, 'storm_status'] = storm_status
            records_df.loc[idx, 'lat'] = convert_lat_lon(lat, 'lat')
            records_df.loc[idx, 'lon'] = convert_lat_lon(lon, 'lon')
            records_df.loc[idx, 'max_wind'] = float(max_wind)
            records_df.loc[idx, 'min_pressure'] = float(min_pressure)

            idx +=1
            
    return records_df


def convert_lat_lon(value, col):
    """
    Lat/lon is encoded with the numeric value plus E/W/N/S. We want to 
    convert this into an absolute decimal value for folium to interpret.
    """
    if col=='lon':
        amount = -float(re.sub('[EW]', '', value)) if 'W' in value else float(re.sub('[EW]', '', value))
    elif col=='lat':
        amount = -float(re.sub('[NS]', '', value)) if 'S' in value else float(re.sub('[NS]', '', value))
    
    return amount

def plot_path(df, folium_map):
    """
    Generates a tuple comprising of lat/lon for the 
    folium map to plot.
    """
    # Folium requires an array of tuples
    points = df[['lat', 'lon']]
    tuple_points = [tuple(x) for x in points.values]
    return tuple_points

def hurr_category(max_wind):
    """
    Defining storm category at a point in time.
    """
    if max_wind <= 73:
        cat = 'TS'
    if (74 <= max_wind <= 95):
        cat = 1
    if (96 <= max_wind <= 110):
        cat = 2
    if (111 <= max_wind <= 129):
        cat = 3
    if (130 <= max_wind <= 156):
        cat = 4
    if max_wind >= 157:
        cat = 5
    return cat

def plot_paths_for_year(df, folium_map):
    """
    Plots hurricane path for all hurricanes in the dataframe
    where lines are color coded for the intensity (category) at that
    point in time.
    """
    # Map colors of path based on category level
    path_colors = {1: '#ffb3b3',
                   2: '#ff8080',
                   3: '#e60000',
                   4: '#ff0000',
                   5: '#580808',
                  'TS': '#ffe6e6'}
    
    for name in df['name'].unique():
        name_df = df[df['name']==name]
        for i, dt in name_df.iterrows():
            day_df = name_df.ix[i:i+1, :]
            day_points = plot_path(day_df, folium_map)
            date = name_df.loc[i, 'date']
            time = name_df.loc[i, 'time']
            category = hurr_category(name_df.loc[i, 'max_wind'])
            maxwind = name_df.loc[i, 'max_wind']
            hurr_info = f"""
            Storm:    {name}
            Date:     {date}
            Time:     {time}
            Category: {category}
            Max Wind: {maxwind}mph
            """
            folium.PolyLine(day_points, tooltip=name, popup=hurr_info, color=path_colors[category]).add_to(folium_map)
            
    return

def get_highest_category(input_df):
    """
    Each row is a hurricane at a point in time. Category will vary
    depending on wind speed at the time. We want to have an identifier
    for what each hurricane's max recorded category was.
    """
    df = input_df.copy(deep=True)
    for hurricane in df['id'].unique():
        hurr_df = df[df['id']==hurricane]
        try:
            max_cat = max(hurr_df[hurr_df['category']!='TS']['category'])
        except ValueError:
            max_cat = 'TS'
        df_indexes = hurr_df.index
        df.loc[df_indexes, 'category_highest'] = max_cat
        
    return df

### Create the hurdat Pandas dataframe
#### TODO: make faster

#### Parse records for entire hurricane data history into dataframe


In [6]:
print(datetime.datetime.now())

2019-06-09 22:40:39.634400


In [7]:
all_records = get_all_hurricanes(records)
hurricanes_df_raw = create_hurricane_df(all_records)

In [13]:
hurricanes_df_raw.to_csv('../data/hurricanes_df_raw.csv', index=False)

#### Add identifier if hurricane made landfall

In [8]:
def over_land(hurricane_df):
    bm = Basemap()
    hurricane_df['over_land'] = np.nan

    for i, row in hurricane_df.iterrows():
        hurricane_df.loc[i, 'over_land'] = np.where(bm.is_land(hurricane_df.loc[i, 'lat'], 
                                                                hurricane_df.loc[i, 'lon'])==True, 
                                                     1, 0)
        
    return hurricane_df

def made_landfall(storm_df):
    storm_df['landfall'] = np.nan # initialize
    storm_df['landfall'] = np.where(storm_df['over_land'].sum()>0, 1, 0)
    
    return storm_df

def storm_duration(storm_df):
    storm_df['duration'] = np.nan # initialize
    start = storm_df['dt'].min()
    end = storm_df['dt'].max()
    storm_df['duration'] = end - start
    
    return storm_df

def is_hurricane(storm_df):
    storm_df['is_hurricane'] = np.nan
    storm_df['is_hurricane'] = np.where(storm_df['storm_status'].any()=="HU", 1, 0)
    
    return storm_df

In [9]:
print(datetime.datetime.now())

2019-06-09 23:07:18.079869


In [14]:
# Determine if hurricane was over land for each data point
all_hurricanes_df = over_land(hurricanes_df_raw)

# Get storm-specific data
hurricanes_cleaned = []
for storm_id in all_hurricanes_df['id'].unique():
    storm_df = all_hurricanes_df[all_hurricanes_df['id']==storm_id]
    
    # Did it make landfall?
    made_landfall(storm_df)
    # How long was it?
    storm_duration(storm_df)
    # Did it reach hurricane status?
    is_hurricane(storm_df)
    
    hurricanes_cleaned.append(storm_df)

all_hurricanes_df_cleaned = pd.concat(hurricanes_cleaned)

# Add year
all_hurricanes_df_cleaned['year'] = pd.DatetimeIndex(all_hurricanes_df_cleaned['dt']).year

C:\Users\Himanshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Himanshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Himanshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [15]:
print(datetime.datetime.now())

2019-06-09 23:30:19.854703


In [16]:
all_hurricanes_df_cleaned.to_csv('../data/hurricanes_df_cleaned.csv', index=False)

#### Load data if above is not run

In [17]:
# Load hurdat data formatted into dataframe
hurricanes_df = pd.read_csv("../data/hurricanes_df_cleaned.csv")
hurricanes_df[hurricanes_df['id']=='AL112017'] # IRMA in 2017

,id,name,date,time,dt,storm_status,lat,lon,max_wind,min_pressure,over_land,landfall,duration,is_hurricane,year
49896,AL112017,IRMA,20170830,0,2017-08-30 00:00:00,TD,16.1,-26.9,30.0,1008.0,1.0,1,14 days 12:00:00.000000000,0,2017
49897,AL112017,IRMA,20170830,600,2017-08-30 06:00:00,TS,16.2,-28.3,35.0,1007.0,1.0,1,14 days 12:00:00.000000000,0,2017
49898,AL112017,IRMA,20170830,1200,2017-08-30 12:00:00,TS,16.3,-29.7,45.0,1006.0,0.0,1,14 days 12:00:00.000000000,0,2017
49899,AL112017,IRMA,20170830,1800,2017-08-30 18:00:00,TS,16.3,-30.8,50.0,1004.0,0.0,1,14 days 12:00:00.000000000,0,2017
49900,AL112017,IRMA,20170831,0,2017-08-31 00:00:00,TS,16.3,-31.7,55.0,999.0,0.0,1,14 days 12:00:00.000000000,0,2017
49901,AL112017,IRMA,20170831,600,2017-08-31 06:00:00,HU,16.4,-32.5,65.0,994.0,0.0,1,14 days 12:00:00.000000000,0,2017
49902,AL112017,IRMA,20170831,1200,2017-08-31 12:00:00,HU,16.7,-33.4,80.0,983.0,0.0,1,14 days 12:00:00.000000000,0,2017
49903,AL112017,IRMA,20170831,1800,2017-08-31 18:00:00,HU,17.1,-34.2,95.0,970.0,0.0,1,14 days 12:00:00.000000000,0,2017
49904,AL112017,IRMA,20170901,0,2017-09-01 00:00:00,HU,17.5,-35.1,100.0,967.0,0.0,1,14 days 12:00:00.000000000,0,2017
49905,AL112017,IRMA,20170901,600,2017-09-01 06:00:00,HU,17.9,-36.1,100.0,967.0,0.0,1,14 days 12:00:00.000000000,0,2017


### Compare hurricane paths in 2000-2017 and 1900-1917

In [20]:
hurr_2000s = hurricanes_df[hurricanes_df['year']>=2008]
hurr_2000s_map = folium.Map(location=[30, -60],
                            zoom_start = 4)

plot_paths_for_year(hurr_2000s, hurr_2000s_map)

# Save as html
hurr_2000s_map.save('../maps/hurricane_paths_2008-2017.html')

C:\Users\Himanshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [21]:
hurr_1900s = hurricanes_df[(hurricanes_df['year']>=1918) & (hurricanes_df['year']<=1927)]
hurr_1900s_map = folium.Map(location=[30, -60],
                            zoom_start = 4)
plot_paths_for_year(hurr_1900s, hurr_1900s_map)

# Save as html
hurr_1900s_map.save('../maps/hurricane_paths_1918-1927.html')

C:\Users\Himanshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


### TODO: 
* Plot all cat5 hurricanes in the last 10 years
* Pct of storms that made landfall (https://gis.stackexchange.com/questions/235133/checking-if-a-geocoordinate-point-is-land-or-ocean)
* Filter by storm for each state hit

# ---------------

## Hypothesis 1
#### Storms are making more frequent landfalls due to increased strength and duration of systems.
Analysis of:
* Number of storms
* Strong storms (hurricane status)
* Number of landfalls
* Average duration of storms

In [22]:
# % of storms to make landfall for each 10 year period from 1917-2017
periods = {0: {'name': '1918-1927',
               'start': 1918,
               'end': 1927},
           1: {'name': '1928-1937',
               'start': 1928,
               'end': 1937},
           2: {'name': '1938-1947',
               'start': 1938,
               'end': 1947},
           3: {'name': '1948-1957',
               'start': 1948,
               'end': 1957},
           4: {'name': '1958-1967',
               'start': 1958,
               'end': 1967},
           5: {'name': '1968-1977',
               'start': 1968,
               'end': 1977},
           6: {'name': '1978-1987',
               'start': 1978,
               'end': 1987},
           7: {'name': '1988-1997',
               'start': 1988,
               'end': 1997},
           8: {'name': '1998-2007',
               'start': 1998,
               'end': 2007},
           9: {'name': '2008-2017',
               'start': 2018,
               'end': 2017}
          }

In [23]:
hurr_1900s = hurricanes_df[(hurricanes_df['year']>=1918) & (hurricanes_df['year']<=1927)]

In [24]:
storm_ids = hurr_1900s['id'].unique()
num_storms = len(storm_ids)
num_storms

71

In [25]:
hurr_1900s

,id,name,date,time,dt,storm_status,lat,lon,max_wind,min_pressure,over_land,landfall,duration,is_hurricane,year
13482,AL011918,UNNAMED,19180801,0,1918-08-01 00:00:00,TS,12.6,-58.5,35.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
13483,AL011918,UNNAMED,19180801,600,1918-08-01 06:00:00,TS,12.8,-60.3,35.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
13484,AL011918,UNNAMED,19180801,1200,1918-08-01 12:00:00,TS,13.0,-62.0,35.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
13485,AL011918,UNNAMED,19180801,1800,1918-08-01 18:00:00,TS,13.2,-63.8,35.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
13486,AL011918,UNNAMED,19180802,0,1918-08-02 00:00:00,TS,13.4,-65.5,35.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
13487,AL011918,UNNAMED,19180802,600,1918-08-02 06:00:00,TS,13.7,-67.3,40.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
13488,AL011918,UNNAMED,19180802,1200,1918-08-02 12:00:00,TS,14.0,-69.0,40.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
13489,AL011918,UNNAMED,19180802,1800,1918-08-02 18:00:00,TS,14.4,-70.8,40.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
13490,AL011918,UNNAMED,19180803,0,1918-08-03 00:00:00,TS,14.9,-72.5,45.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
13491,AL011918,UNNAMED,19180803,600,1918-08-03 06:00:00,TS,15.4,-74.3,45.0,-999.0,0.0,0,6 days 18:00:00.000000000,0,1918
